In [15]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import math
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxnbf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:

def calc_salary(row):
    dct = {
        "YEARLY": 1,
        "MONTHLY": 12,
        "HOURLY": 2000
    }

    mx = row["max_salary"]
    mn = row["min_salary"]
    md = row["med_salary"]

    if not math.isnan(md):
        return dct[row["pay_period"]] * md
    if not math.isnan(mx) and not math.isnan(mn):
        return dct[row["pay_period"]] * (mx + mn)/2

    return None

In [17]:
df = pd.read_csv("job_postings.csv")

In [18]:
df = df.dropna(subset=["description", "pay_period"])

df["annual_salary"] = df.apply(calc_salary, axis=1)
selected_columns = ["annual_salary", "description"]
df = df[selected_columns]
df.shape

(6502, 2)

In [19]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def remove_stopwords_numbers_punctionation_and_lemmatize(desc):

    #remove numbers, and links. Confirm this works
    desc = re.sub('[0-9]+\S+|\s\d+\s|\w+[0-9]+|\w+[\*]+.*|\s[\*]+\s|www\.[^\s]+','', desc)
    words = word_tokenize(desc)
    filtered = list(filter(lambda w: w.lower() not in stop_words, words))
    filtered = list(filter(lambda w: w not in punctuation, filtered))


    # filtered = [lemmatizer.lemmatize(word) for word in filtered]
    # filtered = [stemmer.stem(word) for word in filtered]
    return ' '.join(filtered)

df["description"] = df["description"].apply(remove_stopwords_numbers_punctionation_and_lemmatize)


In [20]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(6502, 2)
(5654, 2)


In [21]:
df.to_csv("cleaned.csv", index=False)